In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import requests
import json
import gzip
import shutil
from textblob import TextBlob

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [55]:
listings = pd.read_csv("data/airbnb/airbnblistings_2024.csv")
listings.columns

C:\Users\rahul\AppData\Local\Temp\ipykernel_19880\212705685.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv("data/airbnb/airbnblistings_2024.csv")


Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [56]:
listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21935608,https://www.airbnb.com/rooms/21935608,20240105143223,2024-01-05,previous scrape,Rental unit in New York · ★4.67 · 1 bedroom · ...,NaN,NaN,https://a0.muscache.com/pictures/55c47994-3f02...,5448955,https://www.airbnb.com/users/show/5448955,Mike,2013-03-13,"New York, NY","I run a sales team for a start up in New York, NY",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/5448955/profi...,https://a0.muscache.com/im/users/5448955/profi...,East Village,1.0,3.0,"['email', 'phone', 'work_email']",t,t,NaN,East Village,Manhattan,40.73179,-73.98679,Entire rental unit,Entire home/apt,3,NaN,1 bath,NaN,1.0,[],NaN,30,1124,30.0,30.0,1124.0,1124.0,30.0,1124.0,NaN,NaN,0,0,0,0,2024-01-05,15,0,0,2017-12-25,2019-08-25,4.67,4.87,4.8,5.0,4.8,5.0,4.4,NaN,f,1,1,0,0,0.20
1,743430766348459803,https://www.airbnb.com/rooms/743430766348459803,20240105143223,2024-01-05,previous scrape,Rental unit in Brooklyn · 1 bedroom · 1 bed · ...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,83422715,https://www.airbnb.com/users/show/83422715,D,2016-07-12,"New York, NY",NaN,NaN,NaN,100%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Little Haiti,1.0,2.0,['phone'],t,t,NaN,Flatbush,Brooklyn,40.65375,-73.95502,Entire rental unit,Entire home/apt,3,NaN,1 bath,NaN,1.0,[],$289.00,30,365,30.0,30.0,365.0,365.0,30.0,365.0,NaN,t,30,60,90,365,2024-01-05,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN
2,4539906,https://www.airbnb.com/rooms/4539906,20240105143223,2024-01-06,previous scrape,Rental unit in New York · 1 bedroom · 1 bed · ...,NaN,NaN,https://a0.muscache.com/pictures/d20c2690-17a3...,23540800,https://www.airbnb.com/users/show/23540800,Carl,2014-11-09,"New York, NY","Originally from the UK, I've lived in New York...",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/23540800/prof...,https://a0.muscache.com/im/users/23540800/prof...,Hell's Kitchen,1.0,1.0,"['email', 'phone']",t,t,NaN,Hell's Kitchen,Manhattan,40.75378,-73.99633,Entire rental unit,Entire home/apt,3,NaN,1 bath,NaN,1.0,[],NaN,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,NaN,0,0,0,0,2024-01-06,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
3,22122723,https://www.airbnb.com/rooms/22122723,20240105143223,2024-01-05,previous scrape,Rental unit in New York · 1 bedroom · 1 bed · ...,NaN,NaN,https://a0.muscache.com/pictures/6a943c46-cb6c...,43719554,https://www.airbnb.com/users/show/43719554,Bersabel,2015-09-08,"New York, NY","I'm a very laid-back, sociable, clean person t...",NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/10dc0...,https://a0.muscache.com/im/pictures/user/10dc0...,NaN,1.0,2.0,"['email', 'phone']"

First we load the data set, then we remove the columns we do not need for our analysis. As shown below, the data set contains multiple columns containing information about the host, which can be retained to help map with reviews dataset if needed. Also, the information about the source, when it was scrapped or the url of the listings are dropped. The name and the description of the individual listing are needed as well as we ultimately aim to predict the pricing. Furthermore, we only keep the columns that are related to price (`price` and `accommodates` as these will be needed to calculate the price per person) or to the location (`neighbourhood_cleansed` and the geolocation). The other columns that are kept either contain critical information to identify the listing (`ìd`) or provide additional information that might influence the price or the review of a location, without it being dependent on the host.

In [57]:
listings_cleaned = listings[["id", "name", "host_id", "host_listings_count", "neighbourhood_cleansed", "neighbourhood_group_cleansed", "latitude", "longitude", "property_type",
                             "room_type", "accommodates", "bathrooms_text", "bedrooms", "beds", "price","number_of_reviews",
                             "review_scores_rating", "review_scores_location", "reviews_per_month"]]

In the subsequent steps below, let us go over each column and preprocess the values accordingly

In [58]:
listings['id'].value_counts()

id
21935608               11
31237124               11
48067323               11
35343378               11
48029069               11
                       ..
1035480653919228703     1
1146109740309514690     1
1024082333469763583     1
674775428767990005      1
1281465359952960262     1
Name: count, Length: 49882, dtype: int64

In [59]:
len(listings['id'].unique())

49882

In [60]:
listings['name'].value_counts()

name
Rental unit in New York · 1 bedroom · 1 bed · 1 bath           4322
Rental unit in Brooklyn · 1 bedroom · 1 bed · 1 bath           2402
Rental unit in Brooklyn · 1 bedroom · 1 bed · 1 shared bath    1393
Rental unit in New York · Studio · 1 bed · 1 bath              1205
Rental unit in New York · 1 bedroom · 1 bed · 1 shared bath    1127
                                                               ... 
Stylish1BRSuiteNearParksSubwayColumbia                            1
Modern  3 bedroom apartment NO cleaning fee                       1
Rental unit in Brooklyn · ★New · Studio · 1 bath                  1
Room near Tompkins Square Park!                                   1
1200-2F MedRoom1                                                  1
Name: count, Length: 63061, dtype: int64

In [61]:
bedrooms = listings['name'].apply(lambda x: re.search(r'(\d+)\s+bedroom', str(x)).group(1) if re.search(r'(\d+)\s+bedroom', str(x)) else 1)
beds = listings['name'].apply(lambda x: re.search(r'(\d+)\s+bed(?!room)s?', str(x)).group(1) if re.search(r'(\d+)\s+bed(?!room)s?', str(x)) else 1)

Lets save the values extracted from names for bedrooms and beds and drop the names column. We can use the list of these values to fill misisng values for bedrooms and beds columns later. We dont need to consider bathrooms as we have a text column exclusive for that

In [62]:
listings.drop('name',axis=1,inplace=True)

In [63]:
listings['host_listings_count'].value_counts()

host_listings_count
1.0      189967
2.0       54724
3.0       25655
4.0       15379
5.0        9787
          ...  
585.0         1
588.0         1
414.0         1
644.0         1
413.0         1
Name: count, Length: 479, dtype: int64

In [64]:
listings['host_listings_count'].isnull().sum()

56

In [65]:
listings['neighbourhood_cleansed'].value_counts()

neighbourhood_cleansed
Bedford-Stuyvesant            30109
Williamsburg                  24587
Midtown                       22712
Harlem                        19707
Bushwick                      17385
                              ...  
New Dorp                         11
Bay Terrace, Staten Island       11
Fort Wadsworth                   11
Chelsea, Staten Island           11
Hollis Hills                      8
Name: count, Length: 225, dtype: int64

In [66]:
listings['neighbourhood_cleansed'].isnull().sum()

0

In [67]:
listings['neighbourhood_group_cleansed'].value_counts()

neighbourhood_group_cleansed
Manhattan        184768
Brooklyn         155000
Queens            62625
Bronx             13947
Staten Island      4025
Name: count, dtype: int64

In [68]:
listings['neighbourhood_group_cleansed'].isnull().sum()

0

In [69]:
listings['property_type'].value_counts()

property_type
Entire rental unit                    173423
Private room in rental unit           117098
Private room in home                   27147
Entire home                            14789
Room in hotel                          14198
Entire condo                           13210
Private room in townhouse              12848
Private room in condo                   5843
Entire loft                             5656
Entire townhouse                        5165
Shared room in rental unit              3862
Entire guest suite                      3598
Entire serviced apartment               3536
Room in boutique hotel                  3518
Private room in serviced apartment      3495
Private room in loft                    2359
Private room in guest suite             1709
Entire place                             847
Private room in bed and breakfast        842
Shared room in home                      805
Entire guesthouse                        653
Room in aparthotel                       

In [70]:
listings['property_type'].isnull().sum()

0

In [71]:
[["room_type", "accommodates", "bathrooms_text", "bedrooms", "beds", "price","number_of_reviews",
                             "review_scores_rating", "review_scores_location", "reviews_per_month"]]

[['room_type',
  'accommodates',
  'bathrooms_text',
  'bedrooms',
  'beds',
  'price',
  'number_of_reviews',
  'review_scores_rating',
  'review_scores_location',
  'reviews_per_month']]

In [72]:
listings['room_type'].value_counts()

room_type
Entire home/apt    222658
Private room       188081
Shared room          5605
Hotel room           4021
Name: count, dtype: int64

In [73]:
listings['room_type'].isnull().sum()

0

In [74]:
listings['accommodates'].value_counts()

accommodates
2     196024
1      75559
4      61962
3      35999
6      19103
5      14280
8       6914
7       3628
10      1828
12      1749
16      1602
9        880
11       267
14       247
15       212
13       111
Name: count, dtype: int64

In [75]:
listings['accommodates'].isnull().sum()

0

In [76]:
bathrooms = listings['bathrooms_text'].apply(lambda x: re.search(r'(\d+)',str(x)).group(1) if re.search(r'(\d+)',str(x)) else 1)

In [77]:
listings['bedrooms'].value_counts()

bedrooms
1.0     207759
2.0      59302
3.0      21528
0.0      20848
4.0       8428
5.0       2748
6.0        739
7.0        244
9.0         99
8.0         55
10.0        20
15.0        10
14.0        10
16.0        10
11.0         4
21.0         2
26.0         2
Name: count, dtype: int64

In [78]:
listings['bedrooms'].isnull().sum()

98557

In [79]:
listings['bedrooms'] = listings['bedrooms'].combine_first(pd.Series(bedrooms))
listings['beds'] = listings['beds'].combine_first(pd.Series(beds))
listings['bathrooms'] = bathrooms

In [80]:
listings.drop('bathrooms_text',axis=1,inplace=True)

In [81]:
listings['price']

0             NaN
1         $289.00
2             NaN
3             NaN
4             NaN
           ...   
420360     $28.00
420361    $475.00
420362    $162.00
420363    $250.00
420364     $52.00
Name: price, Length: 420365, dtype: object

In [82]:
listings['price'].isnull().sum()

157561

In [83]:
listings['number_of_reviews'].value_counts()

number_of_reviews
0       127832
1        40254
2        25029
3        17365
4        13482
         ...  
1066         1
1451         1
441          1
955          1
2165         1
Name: count, Length: 784, dtype: int64

In [84]:
listings['number_of_reviews'].isnull().sum()

0

In [85]:
listings['review_scores_rating'].value_counts()

review_scores_rating
5.00    89814
4.00    11193
4.50    10215
4.67     8205
4.75     6958
        ...  
3.99        1
3.53        1
2.89        1
3.45        1
3.39        1
Name: count, Length: 186, dtype: int64

In [86]:
listings['review_scores_rating'].isnull().sum()

127799

In [87]:
listings[listings['review_scores_rating'].isnull() & listings['price'].isnull()].shape

(51261, 73)

In [88]:
missing_listing_ids = listings[listings['review_scores_rating'].isnull() & listings['price'].isnull()]['id'].unique()

In [89]:
missing_listing_ids

array([            4539906,            22122723,            26295433, ...,
       1253732238224136520, 1254372592138384859, 1257025240961774482],
      dtype=int64)

In [90]:
len(missing_listing_ids)

8107

In [91]:
## Reading Reviews dataset to see if the missing listing ids have reviews or not
filtered_data = []
for chunk in pd.read_csv("data/airbnb/airbnbreviews_2024.csv", chunksize=100000):
    filtered_chunk = chunk[chunk['listing_id'].isin(missing_listing_ids)]
    filtered_data.append(filtered_chunk)

reviews = pd.concat(filtered_data, ignore_index=True)

In [92]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,23252265,244418268,2018-03-18,105719199,Brita,Laura's place is incredibly conveniently locat...
1,23252265,351988678,2018-11-24,26323711,Katherine,The apartment is adorable and extremely conven...
2,23252265,357142886,2018-12-09,136634067,Matthew,Fantastic value. Awesome location and very cut...
3,23252265,430012819,2019-03-29,22656621,Emilee,"Great location, easy access to trains, up abov..."
4,23252265,452126197,2019-05-12,22524456,Scott,"Great location in the west village, near every..."


In [93]:
reviews.shape

(5002, 6)

In [94]:
len(reviews['listing_id'].unique())

381

In [95]:
def rate_review(review):
    polarity = TextBlob(review).sentiment.polarity
    if polarity <= -0.5:
        return 1
    elif -0.5 < polarity <= 0:
        return 2
    elif 0 < polarity <= 0.6:
        return 3
    elif 0.6 < polarity <= 0.9:
        return 4
    else:
        return 5

reviews['rating'] = reviews['comments'].apply(rate_review)
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,rating
0,23252265,244418268,2018-03-18,105719199,Brita,Laura's place is incredibly conveniently locat...,3
1,23252265,351988678,2018-11-24,26323711,Katherine,The apartment is adorable and extremely conven...,3
2,23252265,357142886,2018-12-09,136634067,Matthew,Fantastic value. Awesome location and very cut...,3
3,23252265,430012819,2019-03-29,22656621,Emilee,"Great location, easy access to trains, up abov...",3
4,23252265,452126197,2019-05-12,22524456,Scott,"Great location in the west village, near every...",3


In [96]:
# Removing listings that has no reviews or prices or mentions in reviews dataset

missing_listings = set(missing_listing_ids).difference(set(reviews['listing_id'].unique()))
listings = listings[~listings['id'].isin(missing_listings)].reset_index()

In [97]:
listings.drop('index',axis=1,inplace=True)
listings.head()

,id,listing_url,scrape_id,last_scraped,source,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21935608,https://www.airbnb.com/rooms/21935608,20240105143223,2024-01-05,previous scrape,NaN,NaN,https://a0.muscache.com/pictures/55c47994-3f02...,5448955,https://www.airbnb.com/users/show/5448955,Mike,2013-03-13,"New York, NY","I run a sales team for a start up in New York, NY",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/5448955/profi...,https://a0.muscache.com/im/users/5448955/profi...,East Village,1.0,3.0,"['email', 'phone', 'work_email']",t,t,NaN,East Village,Manhattan,40.73179,-73.98679,Entire rental unit,Entire home/apt,3,1,1,1.0,[],NaN,30,1124,30.0,30.0,1124.0,1124.0,30.0,1124.0,NaN,NaN,0,0,0,0,2024-01-05,15,0,0,2017-12-25,2019-08-25,4.67,4.87,4.80,5.00,4.8,5.0,4.4,NaN,f,1,1,0,0,0.20
1,743430766348459803,https://www.airbnb.com/rooms/743430766348459803,20240105143223,2024-01-05,previous scrape,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,83422715,https://www.airbnb.com/users/show/83422715,D,2016-07-12,"New York, NY",NaN,NaN,NaN,100%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Little Haiti,1.0,2.0,['phone'],t,t,NaN,Flatbush,Brooklyn,40.65375,-73.95502,Entire rental unit,Entire home/apt,3,1,1,1.0,[],$289.00,30,365,30.0,30.0,365.0,365.0,30.0,365.0,NaN,t,30,60,90,365,2024-01-05,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,1,1,0,0,NaN
2,22434861,https://www.airbnb.com/rooms/22434861,20240105143223,2024-01-05,previous scrape,NaN,NaN,https://a0.muscache.com/pictures/3b4347a6-3300...,5925222,https://www.airbnb.com/users/show/5925222,Sophia,2013-04-14,"New York, NY",Female young professional and mother renting o...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/a6386...,https://a0.muscache.com/im/pictures/user/a6386...,NaN,1.0,5.0,"['email', 'phone']",t,t,NaN,Bushwick,Brooklyn,40.69020,-73.91782,Entire rental unit,Entire home/apt,6,2,3,4.0,[],NaN,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,NaN,0,0,0,0,2024-01-05,1,0,0,2018-01-01,2018-01-01,5.00,5.00,5.00,5.00,5.0,5.0,5.0,NaN,f,1,1,0,0,0.01
3,35713272,https://www.airbnb.com/rooms/35713272,20240105143223,2024-01-05,previous scrape,NaN,NaN,https://a0.muscache.com/pictures/fcc9cf1d-b49d...,268631641,https://www.airbnb.com/users/show/268631641,Harris,2019-06-14,NaN,NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/9b079...,https://a0.muscache.com/im/pictures/user/9b079...,Alphabet City,1.0,1.0,['phone'],t,f,NaN,East Village,Manhattan,40.72385,-73.97695,Private room in rental unit,Private room,1,1,1,1.0,[],NaN,30,1125,30.0,30.0,1125.0,1125.0,30.0,1125.0,NaN,NaN,0,0,0,0,2024-01-05,1,0,0,2019-06-20,2019-06-20,5.00,5.00,5.00,5.00,5.0,5.0,5.0,NaN,f,1,0,1,0,0.02
4,583352,https://www.airbnb.com/rooms/583352,202401

In [98]:
# Filtering reviews by 2024 and taking mean of reviews per listing id
reviews = reviews[reviews['date']>'2024-01-01']
ratings_dict = reviews.groupby('listing_id')['rating'].mean().to_dict()

In [99]:
for listing_id in ratings_dict.keys():
    missing_mask = (listings['id'] == listing_id) & (listings['review_scores_rating'].isnull())
    listings.loc[missing_mask, 'review_scores_rating'] = ratings_dict[listing_id]

In [100]:
# Selecting only necessary columns

listings = listings[["id", "host_id", "host_listings_count", "neighbourhood_cleansed", "neighbourhood_group_cleansed", "latitude", "longitude", "property_type",
                             "room_type", "accommodates", "bathrooms", "bedrooms", "beds", "price","number_of_reviews",
                             "review_scores_rating"]]

In [101]:
listings.isnull().sum()

id                                   0
host_id                              0
host_listings_count                 12
neighbourhood_cleansed               0
neighbourhood_group_cleansed         0
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            0
bedrooms                             0
beds                                 0
price                           107009
number_of_reviews                    0
review_scores_rating             51834
dtype: int64

In [102]:
listings['price'] = listings['price'].str.extract('(\d+)', expand=False)
listings.astype({
    "id" : int, "neighbourhood_cleansed": 'category', "latitude": float, "longitude": float, "property_type": 'category',
                             "room_type": 'category', "accommodates": int, "number_of_reviews": int,
                             "review_scores_rating": float, 'bathrooms': float, 'bedrooms': float, 'beds': float})

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\rahul\AppData\Local\Temp\ipykernel_19880\2638930725.py:1: SyntaxWarning: invalid escape sequence '\d'
  listings['price'] = listings['price'].str.extract('(\d+)', expand=False)


,id,host_id,host_listings_count,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating
0,21935608,5448955,1.0,East Village,Manhattan,40.731790,-73.986790,Entire rental unit,Entire home/apt,3,1.0,1.0,1.0,NaN,15,4.67
1,-682893541,83422715,1.0,Flatbush,Brooklyn,40.653750,-73.955020,Entire rental unit,Entire home/apt,3,1.0,1.0,1.0,289,0,NaN
2,22434861,5925222,1.0,Bushwick,Brooklyn,40.690200,-73.917820,Entire rental unit,Entire home/apt,6,2.0,3.0,4.0,NaN,1,5.00
3,35713272,268631641,1.0,East Village,Manhattan,40.723850,-73.976950,Private room in rental unit,Private room,1,1.0,1.0,1.0,NaN,1,5.00
4,583352,2423042,3.0,East Village,Manhattan,40.723260,-73.979690,Entire loft,Entire home/apt,2,1.0,1.0,1.0,NaN,3,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346382,-1425827675,3223938,742.0,Bensonhurst,Brooklyn,40.612310,-73.980300,Private room in rental unit,Private room,1,1.0,4.0,1.0,28,0,NaN
346383,-1988613399,43042852,2.0,Bedford-Stuyvesant,Brooklyn,40.684703,-73.957000,Entire home,Entire home/apt,6,2.0,3.0,3.0,475,0,NaN
346384,1580531882,23297442,1.0,Bedford-Stuyvesant,Brooklyn,40.686559,-73.922157,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,162,0,NaN
346385,325854658,530889967,1.0,Port Morris,Bronx,40.806013,-73.927053,Entire rental unit,Entire home/apt,3,1.0,1.0,1.0,250,0,NaN


In [103]:
listings.loc[(listings['number_of_reviews']==0) & (listings['review_scores_rating'].isnull())]

,id,host_id,host_listings_count,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating
1,743430766348459803,83422715,1.0,Flatbush,Brooklyn,40.653750,-73.955020,Entire rental unit,Entire home/apt,3,1,1,1.0,289,0,NaN
16,1010359952821415638,310205815,4.0,East Village,Manhattan,40.724217,-73.988522,Private room in rental unit,Private room,2,2,1,1.0,100,0,NaN
22,53214934,27672911,1.0,Hell's Kitchen,Manhattan,40.769660,-73.992540,Entire chalet,Entire home/apt,8,1,3,12.0,271,0,NaN
55,799170991396680986,3223938,602.0,Bushwick,Brooklyn,40.701118,-73.920544,Private room in rental unit,Private room,1,1,4,1.0,41,0,NaN
70,31388864,195034974,5.0,Bushwick,Brooklyn,40.696890,-73.925190,Private room in rental unit,Private room,1,1,1,1.0,40,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346382,1281096465598617765,3223938,742.0,Bensonhurst,Brooklyn,40.612310,-73.980300,Private room in rental unit,Private room,1,1,4.0,1.0,28,0,NaN
346383,1281230979116575465,43042852,2.0,Bedford-Stuyvesant,Brooklyn,40.684703,-73.957000,Entire home,Entire home/apt,6,2,3.0,3.0,475,0,NaN
346384,1281263542832791722,23297442,1.0,Bedford-Stuyvesant,Brooklyn,40.686559,-73.922157,Entire rental unit,Entire home/apt,2,1,1.0,1.0,162,0,NaN
346385,1281431427554747842,530889967,1.0,Port Morris,Bronx,40.806013,-73.927053,Entire rental unit,Entire home/apt,3,1,1.0,1.0,250,0,NaN


We can see that all instances of ratings missing is due to the fact that we dont have reviews available. Since we need the data with reviews, we can drop these instances

In [104]:
listings.dropna(subset='review_scores_rating',inplace=True)
listings.isnull().sum()

id                                   0
host_id                              0
host_listings_count                 12
neighbourhood_cleansed               0
neighbourhood_group_cleansed         0
latitude                             0
longitude                            0
property_type                        0
room_type                            0
accommodates                         0
bathrooms                            0
bedrooms                             0
beds                                 0
price                           106994
number_of_reviews                    0
review_scores_rating                 0
dtype: int64

In [105]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294553 entries, 0 to 346232
Data columns (total 16 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            294553 non-null  int64  
 1   host_id                       294553 non-null  int64  
 2   host_listings_count           294541 non-null  float64
 3   neighbourhood_cleansed        294553 non-null  object 
 4   neighbourhood_group_cleansed  294553 non-null  object 
 5   latitude                      294553 non-null  float64
 6   longitude                     294553 non-null  float64
 7   property_type                 294553 non-null  object 
 8   room_type                     294553 non-null  object 
 9   accommodates                  294553 non-null  int64  
 10  bathrooms                     294553 non-null  object 
 11  bedrooms                      294553 non-null  object 
 12  beds                          294553 non-null  ob

In [106]:
## Splitting the data based on missing values of price to be used as a test dataset at a future time

test_listings = listings.loc[listings['price'].isnull()].reset_index(drop=True)
clean_listings = listings.loc[listings['price'].notnull()].reset_index(drop=True)

In [107]:
test_listings.to_csv('data/airbnb/airbnb_testdata.csv')
clean_listings.to_csv('data/airbnb/airbnb_data_processed.csv')